In [1]:
import glob
import os
import shutil
import re
from PIL import Image

In [8]:
evenChars = ["bcfguvyz","89destwx","2367kmqr","0145hjnp"]
oddChars = ["prxz","nqwy","jmtv","hksu","57eg","46df","139c","028b"]
evenColRow = {
    "b": [0,0],
    "c": [1,0],
    "f": [2,0],
    "g": [3,0],
    "u": [4,0],
    "v": [5,0],
    "y": [6,0],
    "z": [7,0],

    "8": [0,1],
    "9": [1,1],
    "d": [2,1],
    "e": [3,1],
    "s": [4,1],
    "t": [5,1],
    "w": [6,1],
    "x": [7,1],

    "2": [0,2],
    "3": [1,2],
    "6": [2,2],
    "7": [3,2],
    "k": [4,2],
    "m": [5,2],
    "q": [6,2],
    "r": [7,2],

    "0": [0,3],
    "1": [1,3],
    "4": [2,3],
    "5": [3,3],
    "h": [4,3],
    "j": [5,3],
    "n": [6,3],
    "p": [7,3],
}

oddColRow = {
    "p": [0, 0],
    "r": [1, 0],
    "x": [2, 0],
    "z": [3, 0],

    "n": [0, 1],
    "q": [1, 1],
    "w": [2, 1],
    "y": [3, 1],

    "j": [0, 2],
    "m": [1, 2],
    "t": [2, 2],
    "v": [3, 2],

    "h": [0, 3],
    "k": [1, 3],
    "s": [2, 3],
    "u": [3, 3],

    "5": [0, 4],
    "7": [1, 4],
    "e": [2, 4],
    "g": [3, 4],

    "4": [0, 5],
    "6": [1, 5],
    "d": [2, 5],
    "f": [3, 5],

    "1": [0, 6],
    "3": [1, 6],
    "9": [2, 6],
    "c": [3, 6],

    "0": [0, 7],
    "2": [1, 7],
    "8": [2, 7],
    "b": [3, 7],
}

gridSizeAtPrecision = {
    1: { "rows": 4, "cols": 8 },
    2: { "rows": 4 * 8, "cols": 8 * 4 },
    3: { "rows": 4 * 8 * 4, "cols": 8 * 4 * 8 },
    4: { "rows": 4 * 8 * 4 * 8, "cols": 8 * 4 * 8 * 4 },
    5: { "rows": 4 * 8 * 4 * 8 * 4, "cols": 8 * 4 * 8 * 4 * 8 },
    6: { "rows": 4 * 8 * 4 * 8 * 4 * 8, "cols": 8 * 4 * 8 * 4 * 8 * 4 },
    7: { "rows": 4 * 8 * 4 * 8 * 4 * 8 * 4, "cols": 8 * 4 * 8 * 4 * 8 * 4 * 8 },
    8: {
        "rows": 4 * 8 * 4 * 8 * 4 * 8 * 4 * 8,
        "cols": 8 * 4 * 8 * 4 * 8 * 4 * 8 * 4,
    },
    9: {
        "rows": 4 * 8 * 4 * 8 * 4 * 8 * 4 * 8 * 4,
        "cols": 8 * 4 * 8 * 4 * 8 * 4 * 8 * 4 * 8,
    },
};

def charAtCell(row, col):
    j, rrow = divmod(row, 8)
    i, ccol = divmod(col, 4)
    c1 = evenChars[j][i]
    c2 = oddChars[rrow][ccol]
    return f'{c1}{c2}'

In [3]:
def compress_png(input_file, output_file):
    """Compresses a PNG image"""
    image = Image.open(input_file)
    image = image.convert("RGB") # Remove alpha channel
    image = image.convert("L") # Convert to grayscale
    image.save(output_file)

In [4]:
# Rename files
folder = "geohash_2p/*"
dest = "topology_2p"
for f in sorted(glob.glob(folder)):
    if os.path.isfile(f):
        match = re.match(r'^.*_(\d+)\.png$', f)
        fid = int(match.groups()[0])  # 1 - 1024 for 2 precision 32 by 32 grid
        col, row = divmod(fid - 1, 32) # fid goes from top to bottom left to right
        compress_png(f, f'{dest}/{charAtCell(row,col)}.png')

In [34]:
def geohashToRowCol(geohash):
    precision = len(geohash)

    if precision == 1:
        return evenColRow[geohash]
        
    [xp, yp] = geohashToRowCol(geohash[0:-1])
    
    if (precision - 1) % 2 == 0:
        [x, y] = evenColRow[geohash[-1]]
        return [x + xp * 8, y + yp * 4]
     
    else:
        [x, y] = oddColRow[geohash[-1]]
        return [x + xp * 4, y + yp * 8]


In [35]:
geohashToRowCol('0p')

[0, 24]